# 有限差分法
在得到偏微分方程$r f=\frac{\partial f}{\partial S_{t}} r S_{t}+\frac{\partial f}{\partial t}+\frac{\partial^{2} f}{2 \partial S_{t}^{2}} S_{t}^{2} \sigma^{2}$之后，f的值也无法被直接解出，此时可通过**网格搜索法**来解决

## 网格搜索法
这里, 我们将时间轴T作为横坐标, 价格轴S作为纵坐标。然后将距离行权时间 $T$ 等分成M份足够短 的时间 $\Delta t=\frac{T}{M}$; 同时设定一个足够大的价格 $S_{\max }$ ，使得 $S_{\max }$ 远远超过行权价K，并且将 $S_{\max }$ 也等分为M份, 使得 $\Delta S_{t}=\frac{S_{t}}{M}$。  

![grid](./note_pic/v2-4f5d55367124bc29a48d1b624febab6f_1440w.jpeg)  

图1红点的坐标即为 $(2 \Delta t, 2 \Delta s)$ 。同时, 横坐标依次为 $(0, \Delta t, 2 \Delta t, \ldots, M \Delta t)$; 纵坐标依次 是 $(0, \Delta S, 2 \Delta S, \ldots, M \Delta S)$ 。  

图1的三个边界条件：(1)紫线; (2)绿线；(3红线。这三个边界条件至关重要。  

如果在Put定价中, (1)表示：股价 $=0$, 紫线区域全部行权, 从而put price=K-0=K; (2)表示: put price $=\max \left(K-j * \Delta S_{t}, 0\right)$ ，j是绿线上的某一点的纵坐标；(3表示：put不会行权, put price $=0$ 。  

如果是在call定价中, (1)表示：股价 $=0$, 紫线区域不行权, 从而call price=0; (2)表示: call price $=\max \left(j * \Delta S_{t}-K, 0\right)$, j是绿线上的某一点的纵坐标; (3)表示：call一定会行 权, call price $=S_{\max }-K$   

在得到了边界条件之后, 我们要求的其实就是 $\mathrm{t}=0$ 的时候, $\mathrm{S}$ 处于不同位置的期权价格, 即为纵坐 标上的每一点。因此, 整个计算是从<mark>右边往左边推</mark>。

### 隐式差分法
看图1中的三角形圈出的4个点。蓝点表示已知，三个橙点表示未知。因此，我们希望通过一个蓝点，推出三个橙点

### Forward approximation
在对网格切分之后, 导数可以通过极限逼近。
那么,   

$\frac{\partial f}{\partial t}=\frac{f_{i+1, j}-f_{i, j}}{\Delta t} \cdots$ 期权价格对时间的导数，  

 $\frac{\partial f}{\partial S_{t}}=\frac{f_{i+1, j}-f_{i-1, j}}{2 \Delta S} \cdots$ 期权价格对价格的一阶导数，  

$\frac{\partial^{2} f}{\partial S^{2}}=\frac{f_{i, j+1}-2 f_{i, j}+f_{i, j-1}}{\Delta S^{2}} \cdots$ 期权价格对价格的二阶导数。
在得到了近似表达以后, 我们就能够得到 $(i, j)=(i \Delta t, j \Delta S)$ 位置的PDE近似表达:
$$
\frac{f_{i+1, j}-f_{i, j}}{\Delta t}+r * j * \Delta S \frac{f_{i, j+1}-f_{i, j-1}}{2 \Delta S}+\frac{1}{2} \sigma^{2}(j * \Delta S)^{2} \frac{f_{i, j+1}-2 f_{i, j}+f_{i, j-1}}{\Delta S^{2}}=r * f_{i, j}
$$
进一步化简:
$$
\begin{aligned}
&f_{i, j}=\underbrace{\left[\frac{1}{2} \Delta t *\left(r * j-\sigma^{2} j^{2}\right)\right]}_{a_{j}} f_{i-1, j-1}+\underbrace{\left[1+\Delta t\left(\sigma^{2} j^{2}+r\right)\right]}_{b_{j}} f_{i-1, j} \\
&+\underbrace{\left[-\frac{1}{2} \Delta t *\left(r j+\sigma^{2} j^{2}\right)\right]}_{c_{j}} f_{i-1, j+1}
\end{aligned}
$$
因此，矩阵形式为：
$$
F_{i}=B F_{i-1}
$$
where
$$
\begin{aligned}
F_{i} &=\left[\begin{array}{c}
f_{i, 1} \\
f_{i, 2} \\
\vdots \\
\vdots \\
f_{i, M-1}
\end{array}\right] \\
B &=\left[\begin{array}{ccccc}
b_{1} & c_{1} & 0 & \cdots & 0 \\
a_{2} & b_{2} & c_{2} & \cdots & 0 \\
0 & a_{3} & b_{3} & \ddots & \vdots \\
\vdots & \vdots & \ddots & \ddots & c_{M-2} \\
0 & 0 & \cdots & a_{M-1} & b_{M-1}
\end{array}\right]
\end{aligned}
$$
网址：[期权定价之有限差分法-zhihu](https://zhuanlan.zhihu.com/p/136027192)

In [1]:
import numpy as np

In [2]:
def get_call_matrix(M, K, delta_S):
    # call的三个边界条件
    # 生成(M+1)*(M+1)的矩阵
    # 此矩阵纵轴代表T，横轴为S
    f_matrx = np.matrix(np.array([0.0]*(M+1)*(M+1)).reshape((M+1, M+1)))
    # 边界条件① S=0的时候，call=0
    f_matrx[:,0] = 0.0
    # 边界条件②：在到期的时候，期权=max(δS*j-K, 0)
    for i in range(M + 1):
        f_matrx[M, i] = float(max(delta_S * i - K, 0))
    # 边界条件③：S=S_max的时候，call=S_max-K
    f_matrx[:,M] = float(S_max - K)
    print("f_matrix shape : ", f_matrx.shape)
    return f_matrx

def get_put_matrix(M, K, delta_S):
    f_matrx = np.matrix(np.array([0.0]*(M+1)*(M+1)).reshape((M+1, M+1)))
    # S=S_max的时候，put=0
    f_matrx[:,M] = 0.0
    # S=0的时候，put=K
    f_matrx[:,0] = float(K)
    # 在到期的时候，期权=max(K-S, 0)
    for i in range(M + 1):
        f_matrx[M, i] = float(max(K - delta_S * i, 0))
    print("f_matrix shape : ", f_matrx.shape)
    return f_matrx

def calculate_coeff(j):

    vj2 = (v * j)**2
    aj = 0.5 * delta_T * (r * j - vj2)
    bj = 1 + delta_T * (vj2 + r)
    cj = -0.5 * delta_T * (r * j + vj2)
    return aj, bj, cj

def get_coeff_matrix(M):
    #计算系数矩阵B
    matrx = np.matrix(np.array([0.0]*(M-1)*(M-1)).reshape((M-1, M-1)))
    a1, b1, c1 = calculate_coeff(1)
    am_1, bm_1, cm_1 = calculate_coeff(M - 1)
    matrx[0,0] = b1
    matrx[0,1] = c1
    matrx[M-2, M-3] = am_1
    matrx[M-2, M-2] = bm_1
    for i in range(2, M-1):
        a, b, c = calculate_coeff(i)
        matrx[i-1, i-2] = a
        matrx[i-1, i-1] = b
        matrx[i-1, i] = c
    print("coeff matrix shape : ",  matrx.shape)
    return matrx

def calculate_f_matrix(flag, M):
    if flag == "call":
        f_matrx = get_call_matrix(M, K, delta_S)
    else:
        f_matrx = get_put_matrix(M, K, delta_S)
    matrx = get_coeff_matrix(M)
    inverse_m = matrx.I
    for i in range(M, 0, -1):
        # 迭代
        Fi = f_matrx[i, 1:M]
        Fi_1 = inverse_m * Fi.reshape((M-1, 1))
        Fi_1 = list(np.array(Fi_1.reshape(1, M-1))[0])
        f_matrx[i-1, 1:M]=Fi_1
    # 这一步取出S_t在网格中的位置，然后抽出结果，即为在该股价的期权价格。
    i = np.round(S/delta_S, 0)
    return f_matrx[0, int(i)]

In [3]:
M = 5000
S = 276.10
r = 0.16/100
T = 58 / 365
v = 0.407530933
K = 230
S_max = 500
delta_T = T/M
delta_S = S_max/M
call = calculate_f_matrix('call',M)
put = calculate_f_matrix('put', M)

f_matrix shape :  (5001, 5001)
coeff matrix shape :  (4999, 4999)
f_matrix shape :  (5001, 5001)
coeff matrix shape :  (4999, 4999)


In [4]:
call

48.767751692758694